In [1]:
import requests
response=requests.get("https://www.balldontlie.io/api/v1/games")
response.json()["data"]

[{'id': 47179,
  'date': '2019-01-30T00:00:00.000Z',
  'home_team': {'id': 2,
   'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'name': 'Celtics'},
  'home_team_score': 126,
  'period': 4,
  'postseason': False,
  'season': 2018,
  'status': 'Final',
  'time': ' ',
  'visitor_team': {'id': 4,
   'abbreviation': 'CHA',
   'city': 'Charlotte',
   'conference': 'East',
   'division': 'Southeast',
   'full_name': 'Charlotte Hornets',
   'name': 'Hornets'},
  'visitor_team_score': 94},
 {'id': 48751,
  'date': '2019-02-09T00:00:00.000Z',
  'home_team': {'id': 2,
   'abbreviation': 'BOS',
   'city': 'Boston',
   'conference': 'East',
   'division': 'Atlantic',
   'full_name': 'Boston Celtics',
   'name': 'Celtics'},
  'home_team_score': 112,
  'period': 4,
  'postseason': False,
  'season': 2018,
  'status': 'Final',
  'time': '     ',
  'visitor_team': {'id': 13,
   'abbreviation': 'LAC',
   'city': 'LA',


Objectives: 
1. Get the probability of win of a matches and compare againist betting company
2. After finished the first objectives, use live api to extract statistic and scarp odds from betting campany


To Do:
1. Get a small portion of data of team statistic ✅
2. Pipeline it ✅
3. Build a deep learning classification ✅
    <br>3.1 train/test? 
4. Get the probability ✅
5. check which team estimation perform the best ✅ (Mavericks)
6. scarp betting odds
7. try to keep scarp and see the changes of probability from time to time in matches
8. update the model to make it accurate

Optional task:
1. download data directly from kaggle
2. use make_pipeline instead
3. I have dropped the opportent team for now and remember to add it back

Updates:
<br>2022-03-24 I realize I cannot use the matching point as one of the metrics as the model realized it determine the result and need to get rid of points per game (maybe I can use player performances to see if it determine the result

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import make_scorer
import pandas as pd

Take only the Mavericks as the home team here as Mavericks are known as rely on statistics

In [3]:
df_full=pd.read_csv("games.csv")
df_full_by_team=df_full[df_full["HOME_TEAM_ID"]==1610612742]
df=df_full_by_team[0:20000]

1. drop NA, unneccesary columns ✅
2. check which columns to pipeline ✅
3. get the team name instead of only id ✅

See if need to add below codes:
<br> def get_team(num):
<br>     return team[team["TEAM_ID"]==num]["NICKNAME"].values[0]
<br> df["HOME_TEAM_NAME"]=np.vectorize(get_team)(df["HOME_TEAM_ID"])
<br> df["HOME_TEAM_NAME"]=np.vectorize(get_team)(df["HOME_TEAM_ID"])

In [4]:
df.dropna(axis=0,inplace=True)
df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home'],axis=1,inplace=True)
team=pd.read_csv("teams.csv")
X=df.drop(["HOME_TEAM_WINS","HOME_TEAM_ID","TEAM_ID_away","PTS_home","PTS_away"],axis=1)
y=df[["HOME_TEAM_WINS"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train_scaled = StandardScaler().fit_transform(X_train)

/tmp/ipykernel_19050/120568555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_19050/120568555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',


Try to use make_pipeline below

In [5]:
"""
preproc_numerical_baseline = make_pipeline(StandardScaler())
preproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preproc_baseline = make_column_transformer(
    (preproc_numerical_baseline, ['PTS_home','FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away']),
    (preproc_categorical_baseline, ["HOME_TEAM_ID","VISITOR_TEAM_ID"]),
    remainder="drop")
preproc_baseline
"""

'\npreproc_numerical_baseline = make_pipeline(StandardScaler())\npreproc_categorical_baseline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))\npreproc_baseline = make_column_transformer(\n    (preproc_numerical_baseline, [\'PTS_home\',\'FG_PCT_home\',\n       \'FT_PCT_home\', \'FG3_PCT_home\', \'AST_home\', \'REB_home\',\n       \'PTS_away\', \'FG_PCT_away\', \'FT_PCT_away\', \'FG3_PCT_away\', \'AST_away\',\n       \'REB_away\']),\n    (preproc_categorical_baseline, ["HOME_TEAM_ID","VISITOR_TEAM_ID"]),\n    remainder="drop")\npreproc_baseline\n'

In [6]:
"""
num_transformer = Pipeline([('scaler', StandardScaler())])
cat_transformer = OneHotEncoder(handle_unknown='ignore',sparse=False)
preprocessor = ColumnTransformer([
    ('num_tr', num_transformer, ['PTS_home','FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away']),
    ('cat_tr', cat_transformer, ["TEAM_ID_away"])],remainder='passthrough')
"""

'\nnum_transformer = Pipeline([(\'scaler\', StandardScaler())])\ncat_transformer = OneHotEncoder(handle_unknown=\'ignore\',sparse=False)\npreprocessor = ColumnTransformer([\n    (\'num_tr\', num_transformer, [\'PTS_home\',\'FG_PCT_home\',\n       \'FT_PCT_home\', \'FG3_PCT_home\', \'AST_home\', \'REB_home\',\n       \'PTS_away\', \'FG_PCT_away\', \'FT_PCT_away\', \'FG3_PCT_away\', \'AST_away\',\n       \'REB_away\']),\n    (\'cat_tr\', cat_transformer, ["TEAM_ID_away"])],remainder=\'passthrough\')\n'

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras import callbacks

2022-03-24 16:55:17.007332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 16:55:17.007367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
def nba_prob_estimate():
    model =Sequential()
    model.add(layers.Dense(20,activation="relu",input_dim=X_train_scaled.shape[-1]))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(8, activation='relu'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [9]:
model = nba_prob_estimate()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

es = callbacks.EarlyStopping(patience=30, restore_best_weights=True)
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test),
    epochs=500, batch_size=32, verbose=0,callbacks=[es])

max(history.history["val_accuracy"])

2022-03-24 16:55:17.828440: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-03-24 16:55:17.828487: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-OQ3QMKM): /proc/driver/nvidia/version does not exist
2022-03-24 16:55:17.828822: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
team_id_list=df_full.TEAM_ID_home.unique()
accuracy={}
team=pd.read_csv("teams.csv")
def get_team(num):
    return team[team["TEAM_ID"]==num]["NICKNAME"].values[0]

In [14]:
for i in team_id_list:
    df_full=pd.read_csv("games.csv")
    df_full_by_team=df_full[df_full["HOME_TEAM_ID"]==i]
    df=df_full_by_team[0:20000]
    df.dropna(axis=0,inplace=True)
    df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',
           'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home'],axis=1,inplace=True)
    team=pd.read_csv("teams.csv")
    X=df.drop(["HOME_TEAM_WINS","HOME_TEAM_ID","TEAM_ID_away","PTS_home","PTS_away"],axis=1)
    y=df[["HOME_TEAM_WINS"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    X_train_scaled = StandardScaler().fit_transform(X_train)
    model = nba_prob_estimate()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    es = callbacks.EarlyStopping(patience=30, restore_best_weights=True)
    history = model.fit(
        X_train, y_train, validation_data=(X_test, y_test),
        epochs=500, batch_size=32, verbose=0,callbacks=[es])
    accuracy[get_team(i)]=max(history.history["val_accuracy"])

/tmp/ipykernel_19050/2127089224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_19050/2127089224.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',
/tmp/ipykernel_19050/2127089224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_19050/2127089224.py:6: SettingWithCopyWarning: 
A val

/tmp/ipykernel_19050/2127089224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_19050/2127089224.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',
/tmp/ipykernel_19050/2127089224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_19050/2127089224.py:6: SettingWithCopyWarning: 
A val

/tmp/ipykernel_19050/2127089224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0,inplace=True)
/tmp/ipykernel_19050/2127089224.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT',


In [15]:
accuracy

{'Heat': 0.8486841917037964,
 'Bulls': 0.7916666865348816,
 'Spurs': 0.8039867281913757,
 'Warriors': 0.7874564528465271,
 'Nuggets': 0.8285714387893677,
 'Jazz': 0.7553957104682922,
 'Trail Blazers': 0.7464285492897034,
 'Magic': 0.8279569745063782,
 'Hawks': 0.7872340679168701,
 'Celtics': 0.8372092843055725,
 'Rockets': 0.7430555820465088,
 'Grizzlies': 0.8505337834358215,
 'Pelicans': 0.7306272983551025,
 'Suns': 0.8056536912918091,
 'Lakers': 0.7354838848114014,
 '76ers': 0.7913669347763062,
 'Hornets': 0.772549033164978,
 'Pistons': 0.7569444179534912,
 'Bucks': 0.8113878965377808,
 'Timberwolves': 0.7397769689559937,
 'Mavericks': 0.8611111044883728,
 'Kings': 0.7174721360206604,
 'Clippers': 0.7735191583633423,
 'Pacers': 0.7622377872467041,
 'Thunder': 0.7447552680969238,
 'Wizards': 0.7692307829856873,
 'Cavaliers': 0.7857142686843872,
 'Raptors': 0.7491165995597839,
 'Nets': 0.8028674125671387,
 'Knicks': 0.7827715277671814}